In [1]:
from fourier_system import *

/Users/nicholaslyu/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [75]:
n = 3 

h = torch.randn((2*n, 2*n)) * 5
h = (h - h.T) / 2
d = torch.randn((2*n)) * 5
# d = torch.zeros_like(d)
d[1] = 1

G2 = torch.randn((2*n, 2*n)) * 5
G2 = (G2 - G2.T) / 2
# G2 = torch.zeros_like(G2)

G1 = torch.randn((2*n)) * 5
# G1 = torch.zeros_like(G1)
G1[0] = 1

Cl = CliffordAlgebra(n, 'cpu')

In [76]:
# Encodes (h, d) into an operator 
def affine_operator_encode(h, d):
    assert torch.allclose(h, -h.transpose(1, 0))
    result = torch.zeros_like(Cl.id())
    for i in range(2*n):
        for j in range(2*n):
            result += h[i][j] * Cl(i+1) @ Cl(j+1) / 2
        result += d[i] * Cl(i+1)
    return result 

# Special encoding into a unitary operator
def affine_unitary_encode(h, d):
    U = torch.matrix_exp(affine_operator_encode(h, d * 1j))
    assert torch.allclose(U @ Adj(U), Cl.id())
    return U 

# Decodes operator into raw (h, d)
def affine_operator_decode(op):
    h = torch.zeros((2*n, 2*n)).type_as(Cl.id())
    d = torch.zeros(2*n).type_as(Cl.id())
    Ivt = IndexMaskCvt(2*Cl.n)
    D = Cl.fourier_coeffs(op)
    for k, v in D.items():
        k = Ivt.m2n(k)
        if len(k) == 1:
            d[k[0] - 1] = v 
        elif len(k) == 2:
            h[k[0] - 1, k[1] - 1] = v 
            h[k[1] - 1, k[0] - 1] = -v 
        else:
            raise RuntimeWarning(f'Decoding encounters non-affine term {k, v} of length {len(k)}')
    return h, d 

def affine_matrix_encode(h, d, unitary_encoding=False):
    E = torch.zeros((2*n+1, 2*n+1)).type_as(Cl.id())
    E[1:, 1:] = h 
    if unitary_encoding:
        E[0, 1:] = E[1:, 0] = -1j * d 
    else:
        E[0, 1:] = -d 
        E[1:, 0] = d 
    return E 

U = affine_unitary_encode(h, d)
op = affine_operator_encode(G2, G1)

Cl.fourier_coeffs(U @ op @ Adj(U))

{(1, 0, 0, 0, 0, 0): tensor(-0.3775-2.8662j, dtype=torch.complex128),
 (0, 1, 0, 0, 0, 0): tensor(0.8093-0.5170j, dtype=torch.complex128),
 (1, 1, 0, 0, 0, 0): tensor(-0.6485+0.0208j, dtype=torch.complex128),
 (0, 0, 1, 0, 0, 0): tensor(-6.0754+2.2692j, dtype=torch.complex128),
 (1, 0, 1, 0, 0, 0): tensor(0.4931-0.2530j, dtype=torch.complex128),
 (0, 1, 1, 0, 0, 0): tensor(1.0791+0.2076j, dtype=torch.complex128),
 (0, 0, 0, 1, 0, 0): tensor(4.2630+1.4728j, dtype=torch.complex128),
 (1, 0, 0, 1, 0, 0): tensor(-1.2310+0.2884j, dtype=torch.complex128),
 (0, 1, 0, 1, 0, 0): tensor(1.1544-0.3833j, dtype=torch.complex128),
 (0, 0, 1, 1, 0, 0): tensor(0.4835+1.7838j, dtype=torch.complex128),
 (0, 0, 0, 0, 1, 0): tensor(-0.7593-2.2623j, dtype=torch.complex128),
 (1, 0, 0, 0, 1, 0): tensor(0.3842+0.0882j, dtype=torch.complex128),
 (0, 1, 0, 0, 1, 0): tensor(2.5627-0.2310j, dtype=torch.complex128),
 (0, 0, 1, 0, 1, 0): tensor(-2.2957+1.9288j, dtype=torch.complex128),
 (0, 0, 0, 1, 1, 0): tensor(

In [81]:
D, E = affine_matrix_encode(G2, G1), affine_matrix_encode(h, d, True)
V = torch.matrix_exp(2 * E)

O = V @ D @ V.T
O[5, 6]

tensor(-0.5211+0.2589j, dtype=torch.complex128)